In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from xgboost import DMatrix
from sklearn.datasets import load_svmlight_file, dump_svmlight_file
from multiprocessing import Pool
import pyltr

c:\users\samve\appdata\local\programs\python\python37\lib\site-packages\sklearn\externals\six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [2]:
df_test = pd.read_csv("data/test_prep_long3-all.csv")
df_train = pd.read_csv("data/train_prep_long3-all.csv")

In [3]:
uninteresting = ["visitor_hist_starrating", "visitor_hist_adr_usd", "booking_bool", "click_bool"]
df_train.drop(uninteresting, axis=1, inplace=True)
uninteresting = ["visitor_hist_starrating", "visitor_hist_adr_usd"]
df_test.drop(uninteresting, axis=1, inplace=True)
# predictors = [c for c in df_train.columns if c not in ["prop_id","srch_id","booking_bool",\
#                                 "click_bool","gross_bookings_usd","position", "category", "visitor_hist_starrating", "visitor_hist_adr_usd"]]
# X_train = df_train[predictors]

    # predicting columns of test set
# cols = [col for col in df_test.columns if col not in ['prop_id', 'srch_id', "visitor_hist_starrating", "visitor_hist_adr_usd"]]
# X_test = df_test[cols]

In [4]:
def format(df, datatype):
    df = df.sort_values(by='srch_id', ascending=False)
    features = df[df.columns.tolist()].values
    qid = df['srch_id'].values
    if datatype == "train":
        target = df['category'].values
    else:
        target = np.zeros(len(df))

    return df, features, qid, target, df.columns.values.tolist()

# train 
train, Xtr, qtr, ytr, feature_labels = format(df_train[df_train.srch_id % 10 != 0], "train")

# validation
vali, Xva, qva, yva, feature_labels = format(df_train[df_train.srch_id % 10 == 0], "train")

# test
test, Xte, qte, yte, feature_labels = format(df_test[df_test.srch_id % 10 == 0], "test")

In [ ]:
comment = ' '.join(map(lambda t: '%d:%s' % t, zip(range(len(feature_labels)), feature_labels)))


def dump(args):
    x, y, filename, query_id, comment = args
    dump_svmlight_file(x, y, filename, query_id=query_id, comment=comment, zero_based=False)

p = Pool()
p.map(dump, ((Xtr, ytr, 'data/train.svmlight', qtr, comment),
             (Xva, yva, 'data/vali.svmlight', qva, comment),
             (Xte, yte, 'data/test.svmlight', qte, comment)))

In [ ]:
nwnw

In [ ]:
metric = pyltr.metrics.NDCG(k=3)

print(qva)

# Only needed if you want to perform validation (early stopping & trimming)
monitor = pyltr.models.monitors.ValidationMonitor(
    Xva, yva, qva, metric=metric, stop_after=25)

model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=1000,
    learning_rate=0.02,
    max_features=0.5,
    query_subsample=0.5,
    max_leaf_nodes=10,
    min_samples_leaf=64,
    verbose=1,
)

model.fit(Xtr, ytr, qtr, monitor=monitor)

In [ ]:
def dump_data(x, y, file_output):
    datasets.dump_svmlight_file(x, y, file_output)
    os.remove("%s_tmp" % file_output)

In [ ]:
#  This script demonstrate how to do ranking with xgboost.train


In [ ]:
group_train = []
with open("mq2008.train.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_train.append(int(line.split("\n")[0]))

group_valid = []
with open("mq2008.vali.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_valid.append(int(line.split("\n")[0]))

group_test = []
with open("mq2008.test.group", "r") as f:
    data = f.readlines()
    for line in data:
        group_test.append(int(line.split("\n")[0]))

In [ ]:


train_dmatrix = DMatrix(Xtr, ytr)
valid_dmatrix = DMatrix(Xva, yva)
test_dmatrix = DMatrix(Xte)

# train_dmatrix.set_group("train")
# valid_dmatrix.set_group("valid")

In [ ]:
params = {'objective': 'rank:ndcg', 'eta': 0.1, 'gamma': 1.0,
          'min_child_weight': 0.1, 'max_depth': 6}
xgb_model = xgb.train(params, train_dmatrix, num_boost_round=4,
                      evals=[(valid_dmatrix, 'validation')])
pred = xgb_model.predict(test_dmatrix)

print(pred)